**1. Mount drive and install libararies**

In [ ]:
!pip install torch torchvision
!pip install transformers==2.10.0
!pip install seqeval
!pip install tensorboardx
!pip install simpletransformers==0.9.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**2. Import libraries**

In [18]:
import pandas as pd
import numpy as np

import requests
import gc
import os
import random

from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, recall_score, precision_score
from scipy.special import softmax

import torch

print("Cuda available" if torch.cuda.is_available() is True else "CPU")
print("PyTorch version: ", torch.__version__)

Cuda available
PyTorch version:  1.10.0+cu111


**3. Read training data**

In [23]:
path = '/content/drive/MyDrive/spap_state/spap_state_vaccine/data/'
df = pd.read_csv(path + 'spap_vaccine_training_vaccine.csv')
df = df[['full_text_final', 'label_final']]

print(len(df))
print(df['label_final'].value_counts(normalize = True))

8400
0    0.909881
1    0.090119
Name: label_final, dtype: float64


**4. Performance metrics**

In [13]:
def report_results(A, B):

  A_name = A.name
  B_name = B.name
    
  df = pd.DataFrame({'A': A,
                     'B': B})
  df = df.dropna()
  A = df['A']
  B = df['B']
  
  prec = precision_score(B, A)
  rec = recall_score(B, A)
  f1 = f1_score(B, A)
  acc = accuracy_score(B, A)

  performance = [prec, rec, f1, acc]

  return performance

**4. Set seed for deterministic modeling**

In [19]:
def set_seed(seed):
  
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  
  np.random.seed(seed)
  
  random.seed(seed)
  
  os.environ['PYTHONHASHSEED'] = str(seed)

**5. Base model with CV (K=5)**

In [ ]:
# hyper-parameters

args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 250,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 2,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': True
   }


# set seed number

set_seed(777)


# cross validate

n = 5
kf = KFold(n_splits = n, random_state = 777, shuffle = True)

cv_results = []
for train_index, val_index in kf.split(df):
  # splitting dataframe 
    train_df = df.iloc[train_index]
    val_df = df.iloc[val_index]
  # defining model
    model = ClassificationModel('bert', 'bert-base-uncased', args = args)
  # train model
    model.train_model(train_df)
  # validate model 
    result, model_outputs, wrong_predictions = model.eval_model(val_df)
    val_df['label_final_BERT'] = np.argmax(model_outputs, axis = 1)
  # performance
    performance = report_results(val_df['label_final_BERT'], val_df['label_final'])
    print(performance)
    cv_results.append(performance) 

# report results 

df_cv_results = pd.DataFrame(cv_results, columns = ['precision', 'recall', 'f-1', 'accuracy'])
print(round(df_cv_results.mean(), 4))
print(round(df_cv_results.std(), 4))

Converting to features started.


  0%|          | 0/6720 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Current iteration:   0%|          | 0/840 [00:00<?, ?it/s]

Running loss: 0.126423

**6. Prediction on new tweets**

In [ ]:
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': False,
   'fp16_opt_level': 'O1',
   'max_seq_length': 250,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 2,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': True
   }

model = ClassificationModel('bert', 'bert-base-uncased', args = args)
model.train_model(df)

predictions, raw_outputs = model.predict()